# VGG
    虽然AlexNet证明深层神经网络卓有成效，但它没有提供一个通用的模板来指导后续的研究人员设计新的网络。 在下面的几个章节中，我们将介绍一些常用于设计深层神经网络的启发式概念。
经典卷积神经网络的基本组成部分是下面的这个序列：
* 带填充以保持分辨率的卷积层；
* 非线性激活函数，如ReLU；
* 汇聚层，如最大汇聚层。

而一个VGG块与之类似，由一系列卷积层组成，后面再加上用于空间下采样的最大汇聚层。在最初的VGG论文中 [Simonyan & Zisserman, 2014]，

## VGG块的组成

* 3x3卷积核、填充为1（保持高度和宽度）的卷积层
* 2x2汇聚窗口、步幅为2（每个块后的分辨率减半）的最大汇聚层。

在下面的代码中，我们定义了一个名为vgg_block的函数来实现一个VGG块。

In [44]:
import torch
import torch
from torchvision import transforms
from  torch.utils import data 
import torchvision
import matplotlib.pyplot as plt

'''
一个VGG块由多个卷积层和一个最大池化层组成。
num_convs：一个块的卷积层数
in_channels：输入通道
out_channel：输出通道
'''
def vgg_block(num_convs,in_channels,out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(torch.nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(torch.nn.ReLU())
        in_channels = out_channels
    layers.append(torch.nn.MaxPool2d(kernel_size=2,stride=2))
    return torch.nn.Sequential(*layers)

vgg_block(2,1,64)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

## VGG网络
    与AlexNet、LeNet一样，VGG网络可以分为两部分：第一部分主要由卷积层和汇聚层组成，第二部分由全连接层组成。
<img src="pic\24.jpg" width="500"/>

* VGG神经网络连接图的几个VGG块（在vgg_block函数中定义）。
* 其中有超参数变量conv_arch，该变量指定了每个VGG块里**卷积层个数**和**输出通道数**。
* 全连接模块则与AlexNet中的相同。

原始VGG网络有5个卷积块，其中前两个块各有一个卷积层，后三个块各包含两个卷积层。 第一个模块有64个输出通道，每个后续模块将输出通道数量翻倍，直到该数字达到512。由于该网络使用8个卷积层和3个全连接层，因此它通常被称为**VGG-11**。

In [45]:
'''块中的卷积层数和输出通道数'''
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [46]:
def VGG(conv_arch):
    conv_blks=[]
    in_channels=1    #图片初始通道数
    #返回多个sequential（块），保存在conv_blks列表中
    for (num_convs,out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs,in_channels,out_channels))
        in_channels=out_channels
    #构建vgg网络
    net=torch.nn.Sequential(
        *conv_blks,
        torch.nn.Flatten(),  #全连接层
        #这里的7需要根据卷积后的像素变化计算得出
        torch.nn.Linear(out_channels*7*7,4096),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(4096,4096),
        torch.nn.ReLU(),
        torch.nn.Dropout(0,5),
        torch.nn.Linear(4096,10)
    )
    return net

## 数据读取

In [48]:
def load_data(batch_size,resize):
    # 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式，
    # 并除以255使得所有像素的数值均在0到1之间
    trans =[transforms.ToTensor()]
    #修改图片大小
    if resize:
        trans.insert(0,transforms.Resize(resize)) 
    trans=transforms.Compose(trans)
    #下载训练数据
    mnist_train = torchvision.datasets.FashionMNIST(
        root="datasets",  #保存的目录
        train=True,       #下载的是训练数据集
        transform=trans,   #得到的是pytorch的tensor，而不是图片
        download=True)  #从网上下载
    #下载测试数据
    mnist_test = torchvision.datasets.FashionMNIST(
        root="datasets", train=False, transform=trans, download=True)
    print(len(mnist_train),len(mnist_test))
    #装载数据
    data_loader_train=data.DataLoader(dataset=mnist_train,
                                                batch_size=batch_size,
                                                shuffle=True)   #数据是否打乱
    data_loader_test=data.DataLoader(dataset=mnist_test,
                                    batch_size=batch_size,
                                    shuffle=True)
    return data_loader_train,data_loader_test

## 模型训练

In [49]:
'''定义预测准确率函数'''
def acc(y_hat,y):
    '''
    :param y_hat: 接收二维张量，例如 torch.tensor([[1], [0]...])
    :param y: 接收二维张量，例如 torch.tensor([[0.1, 0.2, 0.7], [0.8, 0.1, 0.1]...]) 三分类问题
    :return:
    '''
    y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y  #数据类型是否相同
    return float(cmp.type(y.dtype).sum())
    
class Accumulator():
    ''' 对评估的正确数量和总数进行累加 '''
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, item):
        return self.data[item]

'''自定义每个批次训练函数'''
def train_epoch(net,train_iter,loss,optimizer,device):
    #判断是不是pytorch得model，如果是，就打开训练模式，pytorch得训练模式默认开启梯度更新
    if isinstance(net,torch.nn.Module):
        net.train()
    #创建样本累加器【累加每批次的损失值、样本预测正确的个数、样本总数】
    metric = Accumulator(3)  
    for x,y in train_iter:
        x=x.to(device)                            #<----------------------GPU
        y=y.to(device)
        #前向传播获取预测结果
        y_hat=net(x)
        #计算损失
        l=loss(y_hat,y) 
        #判断是pytorch自带得方法还是我们手写得方法（根据不同得方法有不同得处理方式）
        if isinstance(optimizer,torch.optim.Optimizer):
            #梯度清零
            optimizer.zero_grad()
            #损失之求和，反向传播（pytorch自动进行了损失值计算）
            l.backward()
            #更新梯度
            optimizer.step()
            #累加个参数
            metric.add(
                float(l)*len(y),  #损失值总数
                acc(y_hat,y),     #计算预测正确得总数
                y.size().numel()  #样本总数
            )
    #返回平均损失值，预测正确得概率
    return metric[0]/metric[2],metric[1]/metric[2]

'''模型测试'''
def test_epoch(net,test_iter,device):
    if isinstance(net,torch.nn.Module):
        net.eval()  #将模型设置为评估模式
    metric=Accumulator(2)
    for x,y in test_iter:
        x=x.to(device)                            #<----------------------GPU
        y=y.to(device)
        metric.add(
            acc(net.forward(x),y),  #计算准确个数
            y.numel()  #测试样本总数
        )
    return metric[0]/metric[1]

'''正式训练'''
def train_LeNet(num_epochs,trian_iter,test_iter,lr,conv_arch):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    net=VGG(conv_arch)  #VGG网络
    net=net.to(device)  #将网络放在gpu或cpu运行<--------------
    loss_list=[]
    train_acc=[]
    test_acc=[]
    #初始化权重
    def init_weight(m):
        if type(m)==torch.nn.Linear or type(m)==torch.nn.Conv2d:
            torch.nn.init.xavier_normal_(m.weight)
    net.apply(init_weight)
    #损失函数
    loss=torch.nn.CrossEntropyLoss()
    #优化器
    optimizer=torch.optim.SGD(net.parameters(),lr=lr)
    #训练
    for epoch in range(num_epochs):
        #返回平均损失值和正确率
        train_metrics=train_epoch(net,trian_iter,loss,optimizer,device)  #<-----训练
        loss_list.append(train_metrics[0])  #保存loss
        train_acc.append(train_metrics[1])   #保存准确率
        #测试集
        test_metric=test_epoch(net,test_iter,device)     #<-------------测试
        test_acc.append(test_metric)
        print(f"epoch{epoch+1}:loss={train_metrics[0]},train_acc={train_metrics[1]*100:.2f}%,test_acc={test_metric*100:.2f}%")
    
    return loss_list,train_acc,test_acc

'''可视化'''
def draw(num_epochs,loss_list,train_acc,test_acc):
    fig,ax=plt.subplots()   #定义画布
    ax.grid(True)          #添加网格
    ax.set_xlabel("epoch")
    #ax.set_ylim(0,1)

    ax.plot(range(num_epochs),loss_list,label="loss")
    ax.plot(range(num_epochs),train_acc,dashes=[6, 2],label="train")
    ax.plot(range(num_epochs),test_acc,dashes=[6, 2],label="test")
    ax.legend()
    plt.show()

In [ ]:
batch_size=64
num_epochs=10
lr=0.01
#定义每个块的参数（卷积层和输出通道）
conv_arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
#查看网络参数
x=torch.randn(size=(1,1,224,224))
for blk in VGG(conv_arch):
    x=blk(x)
    print(blk.__class__.__name__,'output shape:\t',x.shape)
#数据集
train_iter,test_iter=load_data(batch_size=64,resize=224)
#训练
loss_list,train_acc,test_acc=train_LeNet(num_epochs,train_iter,test_iter,lr,conv_arch)